In [1]:
#import some stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm
import glob
import time
import statsmodels.api as sm
import xgboost as xgb
from sklearn.model_selection import train_test_split #use this?
from scipy import stats

In [2]:
csvPath = 'nba-movement-data/data/dataAndLabel'
#df = pd.read_csv('nba-movement-data\data\converted\0021500001_converted.csv')
df = pd.read_csv(csvPath + '/0021500001_dataAndLabel.csv')

games = os.listdir(csvPath)
all_files = glob.glob(os.path.join(csvPath, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in all_files[0:15])
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

C:\Users\JCHam\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (82,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\JCHam\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DtypeWarning: Columns (82,84) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [3]:
#if we want all columns
pd.set_option('display.max_columns', 26)
df.head()

,game_clock,shot_clock,quarter,game_id,event_id,team_id1,player_id1,x_loc1,y_loc1,speed1,angle1,radius1,team_id2,...,radius11,isTO,isFT,isReb,isFTMade,FTsMade,NonShotPlayerID,NonShotPlayerName,ShotPlayerID,ShotPlayerName,SHOT_TIME,possPoints,isShot
0,720.00,24.00,1,21500001,1,-1,-1,11.4806,419.9969,0.000000,0.000000,6.76567,1610612737,...,0.0,False,0.0,False,0.0,0.0,201143.0,Al Horford,NaN,NaN,NaN,0.0,False
1,719.98,23.90,1,21500001,1,-1,-1,26.8673,357.7526,0.000000,0.000000,6.47278,1610612737,...,0.0,False,0.0,False,0.0,0.0,201143.0,Al Horford,NaN,NaN,NaN,0.0,False
2,719.94,23.86,1,21500001,1,-1,-1,25.1361,357.9653,43.605438,-0.122250,6.09299,1610612737,...,0.0,False,0.0,False,0.0,0.0,201143.0,Al Horford,NaN,NaN,NaN,0.0,False
3,719.90,23.82,1,21500001,1,-1,-1,25.0770,355.3443,65.541656,1.548252,6.16468,1610612737,...,0.0,False,0.0,False,0.0,0.0,201143.0,Al Horford,NaN,NaN,NaN,0.0,False
4,719.86,23.78,1,21500001,1,-1,-1,24.0813,353.8495,44.901597,0.983180,6.11854,1610612737,...,0.0,False,0.0,False,0.0,0.0,201143.0,Al Horford,NaN,NaN,NaN,0.0,False


In [4]:
def getDFReady(df):
    df = df.copy()
    df['possPoints'] = df.apply(lambda row: 0 if pd.isnull(row['possPoints']) else row['possPoints'],axis=1)
    df = df[['shot_clock','x_loc1','y_loc1','speed1','angle1','radius1','x_loc2','y_loc2','speed2','angle2','x_loc3','y_loc3','speed3','angle3',
            'x_loc4','y_loc4','speed4','angle4','x_loc5','y_loc5','speed5','angle5','x_loc6','y_loc6','speed6','angle6',
            'x_loc7','y_loc7','speed7','angle7','x_loc8','y_loc8','speed8','angle8','x_loc9','y_loc9','speed9','angle9',
            'x_loc10','y_loc10','speed10','angle10','x_loc11','y_loc11','speed11','angle11','possPoints']] #assumes player1 is always the ball
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X,y

In [5]:
#input array for when model is ready to train/test

X,y = getDFReady(df)
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=24) #can add shuffle=False to not shuffle

In [6]:
from tensorflow.keras.models import Model

from tensorflow.keras.layers import (
    Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, AvgPool1D, AvgPool2D, Reshape,
    Input, Activation, BatchNormalization, Dense, Add, Lambda, Dropout, LayerNormalization)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping

import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

def crps(y_true, y_pred):
    loss = K.mean(K.sum((K.cumsum(y_pred, axis = 1) - K.cumsum(y_true, axis=1))**2, axis=1))/199
    return loss

C:\Users\JCHam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\JCHam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\JCHam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\JCHam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [ ]:
def 